In [34]:
import pandas as pd
customer_master = pd.read_csv('1/customer_master.csv')
item_master = pd.read_csv('1/item_master.csv')
transaction_1 = pd.read_csv('1/transaction_1.csv')
transaction_2 = pd.read_csv('1/transaction_2.csv')
transaction = pd.concat([transaction_1, transaction_2], ignore_index=True)
transaction_detail_1 = pd.read_csv('1/transaction_detail_1.csv')
transaction_detail_2 = pd.read_csv('1/transaction_detail_2.csv')
transaction_details = pd.concat([transaction_detail_1, transaction_detail_2], ignore_index=True)

join_data = pd.merge(transaction_details, transaction[['transaction_id', 'payment_date', 'customer_id']], on='transaction_id', how='left')
join_data = pd.merge(join_data, customer_master, on='customer_id', how='left')
join_data = pd.merge(join_data, item_master, on='item_id', how='left')

KeyError: 'price'

In [44]:
join_data['price'] = join_data['item_price'] * join_data['quantity']

In [45]:
# check sums
print(join_data['price'].sum())
print(transaction['price'].sum())
print(join_data['price'].sum() == transaction['price'].sum())

971135000
971135000
True


In [ ]:
print(join_data.isnull().sum())
print(join_data.describe())
print(f"min: {join_data['payment_date'].min()}")
print(f"max: {join_data['payment_date'].max()}")


In [46]:
print(join_data.dtypes)

join_data['payment_date'] = pd.to_datetime(join_data['payment_date'])
join_data['payment_month'] = join_data['payment_date'].dt.strftime('%Y%m')
join_data[['payment_date', 'payment_month']].head(5)

detail_id                      int64
transaction_id                object
item_id                       object
quantity                       int64
payment_date          datetime64[ns]
customer_id                   object
customer_name                 object
registration_date             object
customer_name_kana            object
email                         object
gender                        object
age                            int64
birth                         object
pref                          object
item_name                     object
item_price                     int64
payment_month                 object
price                          int64
dtype: object


,payment_date,payment_month
0,2019-02-01 01:36:57,201902
1,2019-02-01 01:37:23,201902
2,2019-02-01 02:34:19,201902
3,2019-02-01 02:47:23,201902
4,2019-02-01 04:33:46,201902


In [47]:
join_data.groupby('payment_month').sum()

,detail_id,quantity,age,item_price,price
payment_month,,,,,
201902,676866,1403,59279,142805000,160185000
201903,2071474,1427,58996,142980000,160370000
201904,3476816,1421,59246,143670000,160510000
201905,4812795,1390,58195,139655000,155420000
201906,6369999,1446,61070,147090000,164030000
201907,8106846,1485,62312,153215000,170620000


In [49]:
join_data.groupby(['payment_month', 'item_name']).sum()[['price', 'quantity']]

pd.pivot_table(join_data, index='item_name', columns=['payment_month'], values=['price', 'quantity'], aggfunc='sum')

price                                                    \
payment_month    201902    201903    201904    201905    201906    201907   
item_name                                                                   
PC-A           24150000  26000000  25900000  24850000  26000000  25250000   
PC-B           25245000  25500000  23460000  25330000  23970000  28220000   
PC-C           19800000  19080000  21960000  20520000  21840000  19440000   
PC-D           31140000  25740000  24300000  25920000  28800000  26100000   
PC-E           59850000  64050000  64890000  58800000  63420000  71610000   

              quantity                                     
payment_month   201902 201903 201904 201905 201906 201907  
item_name                                                  
PC-A               483    520    518    497    520    505  
PC-B               297    300    276    298    282    332  
PC-C               165    159    183    171    182    162  
PC-D               173    143    135    144    160    145  
PC-E               285    305    309    280    302    341